In [6]:
# import the libraries
import cv2
import numpy as np
import pylab
from cudamodules import StereoMatchingBasic

In [ ]:
img1 = cv2.imread('Data/leftTest.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('Data/rightTest.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
pylab.imshow(img1, cmap=pylab.gray())
pylab.show()
pylab.imshow(img2, cmap=pylab.gray())
pylab.show()

print(img2.shape)
print(np.min(img2))
print(np.max(img2))
matches = StereoMatchingBasic.stereo_matching_basic(img1, img2)
pylab.imshow(matches, cmap=pylab.gray())
pylab.show()
print(matches.shape)
print(np.min(matches))
print(np.max(matches))